# Amharic Llama 3 Fine Tune
<strong>Notebook Objective</strong>: Qlora fine tune llama 3 on Amharic alpaca using unsloth

In [ ]:
!pip install wandb

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = False # Update to reduce RAM

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: You passed in `unsloth/llama-3-8b-bnb-4bit` which is a 4bit model, yet you set
`load_in_4bit = False`. We shall load `unsloth/llama-3-8b` instead.


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep

In [ ]:
# Prompt format from https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
alpaca_prompt = """<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>

Below is an instruction that describes a task in Amharic. Write a response in Amharic that appropriately completes the request.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|>
<|end_of_text|>
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs       = examples["chosen"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("iocuydi/amharic-alpaca", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from datasets import Dataset

validation_split = int(0.95 * len(dataset))

train_dataset = Dataset.from_dict(dataset[:validation_split])
validation_dataset = Dataset.from_dict(dataset[validation_split:])

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
import wandb
wandb.login()

wandb.init(
    project="am_llama",
    entity="simonbutt",
    name="am_llama3_alpaca_2",
)

wandb.watch(model)

wandb: Currently logged in as: simonbutt. Use `wandb login --relogin` to force relogin


[]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.integrations import WandbCallback

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset= validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    callbacks=[
        WandbCallback()
    ],
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 0,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        eval_steps=30,
        save_steps=60,
        evaluation_strategy="steps",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    )
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/44254 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2330 [00:00<?, ? examples/s]

You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
WandbCallback


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
15.24 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 44,254 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 8
\        /    Total batch size = 64 | Total steps = 691
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
30,0.710500,0.695805
60,0.648500,0.632431
90,0.609400,0.604491
120,0.581500,0.583735
150,0.571400,0.567151
180,0.553600,0.554197
210,0.541100,0.542327
240,0.531700,0.532857
270,0.527700,0.523546
300,0.522700,0.515688


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

34176.5263 seconds used for training.
569.61 minutes used for training.
Peak reserved memory = 37.266 GB.
Peak reserved memory for training = 22.026 GB.
Peak reserved memory % of max memory = 94.192 %.
Peak reserved memory for training % of max memory = 55.672 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "ኃይሌ 10 ፖም አለው እና 2 ይሰጣል። ስንት ተረፈ?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\n\nBelow is an instruction that describes a task in Amharic. Write a response in Amharic that appropriately completes the request.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nኃይሌ 10 ፖም አለው እና 2 ይሰጣል። ስንት ተረፈ?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n<|eot_id|>\n<|end_of_text|>\nኤንቲግራይት ተረፈው ኃይል 10 ፖም አለው እና 2 ይሰጣል። ስንት ተረፈ? ለመፍታት የሚከተለውን ስልተ ቀመር መጠቀም እንችላለን፡- የተረፈው ኃይል = የተረፈው ኃይል አሁን ያለው ኃይል = የተረፈው ኃይል - የተረፈው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን ያለው ኃይል አሁን �']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.push_to_hub("simonbutt/am_llama3_alpaca", token = "") # Online saving

README.md:   0%|          | 0.00/664 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/simonbutt/am_llama3_alpaca


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\n\nBelow is an instruction that describes a task in Amharic. Write a response in Amharic that appropriately completes the request.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nWhat is a famous tall tower in Paris?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n<|eot_id|>\n<|end_of_text|>\nኤንስ፡ በፓሪስ ውስጥ አንድ ታዋቂ ትልቅ ጫ�']

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
model.push_to_hub_merged("simonbutt/am_llama3_alpaca", tokenizer, save_method = "merged_16bit", token = "")


Unsloth: You are pushing to hub, but you passed your HF username = simonbutt.
We shall truncate simonbutt/am_llama3_alpaca to am_llama3_alpaca
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 59.91 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 78.87it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/simonbutt/am_llama3_alpaca


### GGUF / llama.cpp Conversion

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to q4_k_m GGUF
model.push_to_hub_gguf("simonbutt/am_llama3_alpaca", tokenizer, quantization_method = "q4_k_m", token = "")